In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
data = pd.read_csv('../input/bank-note-authentication-uci-data/BankNote_Authentication.csv')

In [ ]:
data

In [ ]:
data.isnull().sum()

In [ ]:
data.corr()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=42)

In [ ]:
print(x_train.shape)
print(x_test.shape)

# XGBOOST 

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(x_train,y_train)

In [ ]:
y_predi = classifier.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predi)

In [ ]:
accuracy_on_train = classifier.score(x_train, y_train)
accuracy_on_test = classifier.score(x_test, y_test)
accuracy_on_train, accuracy_on_test

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predi))

# Randomforest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100, random_state = 20)
rf.fit(x_train, y_train)

In [ ]:
 y_predict = rf.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)


In [ ]:
accuracy_on_train = rf.score(x_train, y_train)
accuracy_on_test = rf.score(x_test, y_test)
accuracy_on_train, accuracy_on_test

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predict))

In [ ]:
rf.predict([[2,3,4,1]])

# PICKLE FILE

In [ ]:
import pickle
pickle_out = open("model_trained.pkl","wb")
pickle.dump(rf, pickle_out)
pickle_out.close()

In [ ]:
rf.predict([[2,3,4,1]])

# the above code for model training
# now below code for deployment

In [ ]:
!pip install flasgger

In [ ]:
import pandas as pd 
import pickle
from flask import Flask, request
import flasgger
from flasgger import Swagger

In [ ]:
app = Flask(__name__)
Swagger(app)
pickle_data = open("./model_trained.pkl","rb")
trained_data = pickle.load(pickle_data)

In [ ]:
@app.route('/')
def welcome():
    return "hello I'm kaggle"

In [ ]:
@app.route('/prediction',methods=["Get"])
def predict_by_single_data_GET_method():
    
    """Let's Authenticate the Banks Note .
    ---
    parameters:  
      - name: variance
        in: query
        type: number
        required: true
      - name: skewness
        in: query
        type: number
        required: true
      - name: curtosis
        in: query
        type: number
        required: true
      - name: entropy
        in: query
        type: number
        required: true
    responses:
        200:
            description: The output values
        
    """
    variance=request.args.get("variance")
    skewness=request.args.get("skewness")
    curtosis=request.args.get("curtosis")
    entropy=request.args.get("entropy")
    prediction=trained_data.predict([[variance,skewness,curtosis,entropy]])
    print(prediction)
    return "This is your prediction = "+str(prediction)

In [ ]:
@app.route('/prediction_with_file',methods=["POST"])
def predict_by_file_POST_method():
    """Let's Authenticate the Banks Note .
    ---
    parameters:
      - name: file
        in: formData
        type: file
        required: true
      
    responses:
        200:
            description: output values
        
    """
    df_test=pd.read_csv(request.files.get("file"))
    print(df_test.head())
    prediction=trained_data.predict(df_test)
    
    return str(list(prediction))

In [ ]:
#if __name__=='__main__':
#    app.run()

** you need to paste this in your browser
#http://127.0.0.1:5000/apidocs/**